In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("hw02.ipynb")

# HW 02: Food Safety

You must submit this assignment to Gradescope by the on-time deadline. Please read the syllabus for the grace period policy. No late submissions beyond the grace period will be accepted. While course staff is happy to help you if you encounter difficulties with submission, we may not be able to respond to late-night requests for assistance. **We strongly encourage you to plan to submit your work to Gradescope several hours before the stated deadline.** This way, you will have ample time to contact staff for submission support. 

After completing this assignment ("hw02"), please submit the generated zip file to the Homework 02 Coding assignment on Gradescope. Gradescope will automatically submit a PDF of your written responses to the HW 02 Written assignment; there is no need to submit it manually. 

## HW Objectives 

In this homework, we will investigate restaurant food safety scores for restaurants in San Francisco. The scores and violation information have been [made available by the San Francisco Department of Public Health](https://data.sfgov.org/Health-and-Social-Services/Restaurant-Scores-LIVES-Standard/pyih-qa8i). The main goal for this assignment is to walk through the process of Data Cleaning and familiarise yourself with some of the `pandas` functions discussed in Pandas I and II. 


After this homework, you should be comfortable with:
* Reading CSV files 
* Reading `pandas` documentation and using `pandas`
* Working with data at different levels of granularity

## Before You Start

For each question in the assignment, please write down your answer in the answer cell(s) right below the question. 

We understand that it is helpful to have extra cells breaking down the process towards reaching your final answer. If you happen to create new cells below your answer to run codes, **NEVER** add cells between a question cell and the answer cell below it. It will cause errors when we run the autograder, and it will sometimes cause a failure to generate the PDF file.

**Important note: The local autograder tests will not be comprehensive. You can pass the automated tests in your notebook but still fail tests in the autograder.** Please be sure to check your results carefully.

Finally, unless we state otherwise, **do not use for loops or list comprehensions**. The majority of this assignment can be done using built-in commands in `pandas` and `NumPy`.  Our autograder isn't smart enough to check, but you're depriving yourself of key learning objectives if you write loops / comprehensions, and you also won't be ready for the midterm.

The cell below imports all the necessary libraries you need to use during this homework. Without running this cell, you will not be able to call the various `NumPy` and `pandas` functions we use later on, so please make sure you run it before starting to work on the homework.

In [ ]:
import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
plt.style.use('fivethirtyeight')

<br/><br/>
<hr style="border: 5px solid #8a8c8c;" />
<hr style="border: 1px solid #ffcd00;" />

# Part 0: Obtaining Data

## File Systems and I/O

In general, we will focus on using Python commands to investigate files.  However, it can sometimes be easier to use shell commands in your local operating system.  The following cells demonstrate how to do this.

In [ ]:
from pathlib import Path
data_dir = Path('.')
data_dir.mkdir(exist_ok = True)
file_path = data_dir / Path('data.zip')
dest_path = file_path

After running the cell above, if you list the contents of the directory containing this notebook, you should see `data.zip.gz`.

*Note*: The command below starts with an `!`. This tells our Jupyter Notebook to pass this command to the operating system. In this case, the command is the `ls` Unix command which lists files in the current directory.

In [ ]:
!ls

## Loading Food Safety Data

We have data, but we don't have any specific questions about the data yet. Let's focus on understanding the structure of the data; this involves answering questions such as:

* Is the data in a standard format or encoding?
* Is the data organized in records?
* What are the fields in each record? (We sometimes also use the term 'feature' or 'attribute' as well, depending on the context)

Let's start by looking at the contents of `data.zip`. It's not just a single file but rather a compressed directory of multiple files. We could inspect it by uncompressing it using a shell command such as `!unzip data.zip`, but in this homework, we're going to do almost everything in Python for maximum portability.

## Looking Inside and Extracting the Zip Files

The following code blocks are for setup. Simply run the cells; **do not modify them**. Question 1a is where you will start to write code.

Here, we assign `my_zip` to a `zipfile.Zipfile` object representing `data.zip`, and assign `list_names` to a list of all the names of the contents in `data.zip`.

In [ ]:
import zipfile
my_zip = zipfile.ZipFile(dest_path, 'r')
list_names = my_zip.namelist()
list_names

You may notice that we did not write `zipfile.ZipFile('data.zip', ...)`. Instead, we used `zipfile.ZipFile(dest_path, ...)`. In general, we **strongly suggest having your filenames hard coded as string literals only once** in a notebook. It is very dangerous to hardcode things twice because if you change one but forget to change the other, you can end up with bugs that are very hard to find.

Now, we display the files' names and their sizes.

In [ ]:
my_zip = zipfile.ZipFile(dest_path, 'r')
for info in my_zip.infolist():
    print('{}\t{}'.format(info.filename, info.file_size))

Often when working with zipped data, we'll never unzip the actual zip file. This saves space on our local computer. However, for this homework the files are small, so we're just going to unzip everything. This has the added benefit that you can look inside the CSV files using a text editor, which might be handy for understanding the structure of the files. The cell below will unzip the CSV files into a sub-directory called `data`.

In [ ]:
data_dir = Path('.')
my_zip.extractall(data_dir)
!ls {data_dir / Path("data")}

The cell above created a folder called `data`, and in it there should be five CSV files. Let's open up `legend.csv` to see its contents. To do this, click on the file icon on the top left to show the folders and files within the hw02 folder, then click on `legend.csv`. The file will open up in another tab. You should see something that looks like:

    "Minimum_Score","Maximum_Score","Description"
    0,70,"Poor"
    71,85,"Needs Improvement"
    86,90,"Adequate"
    91,100,"Good"

The `legend.csv` file does indeed look like a well-formed CSV file. Let's check the other three files. Rather than opening up each file manually, let's use Python to print out the first 5 lines of each. We defined a helper function for you that will allow you to retrieve the first N lines of a file as a list. For example, `head('data/legend.csv', 5)` will return the first 5 lines of "data/legend.csv". Run the cell below to print out the first 5 lines of all six files that we just extracted from the zip file.

In [ ]:
import os

def head(filename, lines=5):
    """
    Returns the first few lines of a file.
    
    filename: the name of the file to open
    lines: the number of lines to include
    
    return: A list of the first few lines from the file.
    """
    from itertools import islice
    with open(filename, "r") as f:
        return list(islice(f, lines))

data_dir = "./"
for f in list_names:
    if not os.path.isdir(f):
        print(head(data_dir + f, 5), "\n")

## Reading in and Verifying Data

Based on the above information, let's attempt to load `bus.csv`, `ins2vio.csv`, `ins.csv`, and `vio.csv` into `pandas` `DataFrame`s with the following names: `bus`, `ins2vio`, `ins`, and `vio`, respectively.

*Note:* Because of character encoding issues, one of the files (`bus`) will require an additional argument `encoding='ISO-8859-1'` when calling `pd.read_csv`. At some point in your future, you should read all about [character encodings](https://diveintopython3.problemsolving.io/strings.html). We won't discuss these in detail in DATA 2201.

In [ ]:
# Path to the directory containing data
dsDir = Path('data')

bus = pd.read_csv(dsDir/'bus.csv', encoding='ISO-8859-1')
ins2vio = pd.read_csv(dsDir/'ins2vio.csv')
ins = pd.read_csv(dsDir/'ins.csv')
vio = pd.read_csv(dsDir/'vio.csv')

# This code is essential for the autograder to function properly. Do not edit
ins_test = ins

Now that you've read the files, let's try some `pd.DataFrame` methods ([docs](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html)).
Use the `DataFrame.head` method to show the top few lines of the `bus`, `ins`, and `vio` `DataFrame`s. For example, running the cell below will display the first few lines of the `bus` `DataFrame`. 

In [ ]:
bus.head()

To show multiple return outputs in one single cell, you can use `display()`. 

In [ ]:
display(bus.head())
display(ins.head())

Now, we perform some sanity checks for you to verify that the data was loaded with the correct structure.

First, we check the basic structure of the `DataFrame`s you created:

In [ ]:
assert all(bus.columns == ['business id column', 'name', 'address', 'city', 'state', 'postal_code',
                           'latitude', 'longitude', 'phone_number'])
assert 6250 <= len(bus) <= 6260

assert all(ins.columns == ['iid', 'date', 'score', 'type'])
assert 26660 <= len(ins) <= 26670

assert all(vio.columns == ['description', 'risk_category', 'vid'])
assert 60 <= len(vio) <= 65

assert all(ins2vio.columns == ['iid', 'vid'])
assert 40210 <= len(ins2vio) <= 40220

Next we'll check that the statistics match what we expect. The following are hard-coded statistical summaries of the correct data.

In [ ]:
bus_summary = pd.DataFrame(**{'columns': ['business id column', 'latitude', 'longitude'],
 'data': {'business id column': {'50%': 75685.0, 'max': 102705.0, 'min': 19.0},
  'latitude': {'50%': -9999.0, 'max': 37.824494, 'min': -9999.0},
  'longitude': {'50%': -9999.0,
   'max': 0.0,
   'min': -9999.0}},
 'index': ['min', '50%', 'max']})

ins_summary = pd.DataFrame(**{'columns': ['score'],
 'data': {'score': {'50%': 76.0, 'max': 100.0, 'min': -1.0}},
 'index': ['min', '50%', 'max']})

vio_summary = pd.DataFrame(**{'columns': ['vid'],
 'data': {'vid': {'50%': 103135.0, 'max': 103177.0, 'min': 103102.0}},
 'index': ['min', '50%', 'max']})

from IPython.display import display

print('What we expect from your Businesses DataFrame:')
display(bus_summary)
print('What we expect from your Inspections DataFrame:')
display(ins_summary)
print('What we expect from your Violations DataFrame:')
display(vio_summary)

The code below defines a testing function that we'll use to verify that your data has the same statistics as what we expect. Run these cells to define the function. The `df_allclose` function has this name because we are verifying that all of the statistics for your `DataFrame` are close to the expected values. Why not `df_allequal`? It's a bad idea in almost all cases to compare two floating point values like 37.780435, as rounding errors can cause spurious failures. Run the following cells to load some basic utilities (you do not need to change these at all):

In [ ]:
"""Run this cell to load this utility comparison function that we will use in various
tests below (both tests you can see and those we run internally for grading).

Do not modify the function in any way.
"""


def df_allclose(actual, desired, columns=None, rtol=5e-2):
    """Compare selected columns of two Dataframes on a few summary statistics.
    
    Compute the min, median and max of the two Dataframes on the given columns, and compare
    that they match numerically to the given relative tolerance.
    
    If they don't match, an AssertionError is raised (by `numpy.testing`).
    """    
    # Summary statistics to compare on
    stats = ['min', '50%', 'max']
    
    # For the desired values, we can provide a full DF with the same structure as
    # the actual data, or pre-computed summary statistics.
    # We assume a pre-computed summary was provided if column is None. In that case, 
    # `desired` *must* have the same structure as the actual's summary
    if columns is None:
        des = desired
        columns = desired.columns
    else:
        des = desired[columns].describe().loc[stats]

    # Extract summary stats from actual DF
    act = actual[columns].describe().loc[stats]

    return np.allclose(act, des, rtol)

We will now explore each file in turn, including determining its granularity and exploring many of the variables individually. Let's begin with the businesses file, which has been read into the `bus` `DataFrame`.

<br/><br/>
<hr style="border: 5px solid #8a8c8c;" />
<hr style="border: 1px solid #ffcd00;" />

# Part 1: Examining the Business Data File


<br/>

---

## Question 1.1

From its name alone, we expect the `bus.csv` file to contain information about the restaurants. Let's investigate the granularity of this dataset.

In [ ]:
bus.head()

The `bus` `DataFrame` contains a column called `business id`, which probably corresponds to a unique business id.  However, we will first rename that column to `bid` for simplicity.

**Note**: In practice, we might want to do this renaming when the table is loaded, but for grading purposes, we will do it here.

In [ ]:
bus = ...
bus.head()

In [ ]:
grader.check("q11")

<br/>

## Cleaning the Business Data Postal Codes

The business data contains postal code information that we can use to aggregate the ratings over regions of the city. Let's examine and clean the postal code field. The postal code (sometimes also called a [ZIP code](https://en.wikipedia.org/wiki/ZIP_Code)) partitions the city into regions:

<img src="https://www.usmapguide.com/wp-content/uploads/2019/03/printable-san-francisco-zip-code-map.jpg" alt="ZIP Code Map" style="width: 600px">

<br/>

---

## Question 1.2

How many restaurants are in each ZIP code? 

In the cell below, create a **Series** where the index is the postal code and the value is the number of records with that postal code. The Series should be in descending order of count. Do you notice any odd/invalid ZIP codes?

**Hint 1**: You may find [value_counts](https://pandas.pydata.org/docs/reference/api/pandas.Series.value_counts.html) helpful.

In [ ]:
zip_counts = ...
print(zip_counts.to_string())

In [ ]:
grader.check("q12")

<br/>

---

## Question 1.3

In the above question, we noticed a large number of potentially invalid ZIP codes (e.g., "Ca"). These are likely due to data entry errors. To get a better understanding of the potential errors in the zip codes, let's break down the problem into two parts.

First, we will import a list of valid San Francisco ZIP codes by using `pd.read_json` to load the file `data/sf_zipcodes.json`, and store them as a Series in `valid_zips`. As you may expect, `pd.read_json` works similarly to `pd.read_csv` but for JSON files (a different file format you'll learn more about later in class) that you can read more about [here](https://pandas.pydata.org/docs/reference/api/pandas.read_json.html). If you are unsure of what data type a variable is, remember you can do `type(some_var_name)` to check!

In [ ]:
valid_zips = pd.read_json('data/sf_zipcodes.json')['zip_codes']
valid_zips

Observe that `pd.read_json` reads data as integers by default. This isn't quite what we want! We would like to store ZIP codes as strings (you'll learn more about why soon!). To do that, we can use the `astype` function to generate a copy of the `pandas` `Series` stored as strings instead.

In [ ]:
valid_zips = valid_zips.astype("string")

In [ ]:
type(valid_zips.dtype)

<!-- BEGIN QUESTION -->

You will probably want to use the `Series.isin` function. For more information on this function see the [the documentation linked in this internet search](https://www.google.com/search?q=series+isin+pandas&rlz=1C1CHBF_enUS910US910&oq=series+isin+pandas&aqs=chrome..69i57l2j69i59j69i60l2j69i65j69i60l2.1252j0j7&sourceid=chrome&ie=UTF-8). 

**Note:** You are welcome and, in fact, encouraged to search and read documentation on the internet to complete the assignments in the course, even if the documentation is not linked explicitly.

In [ ]:
...
invalid_zip_bus = ...
invalid_zip_bus.head(10)

In [ ]:
grader.check("q13")

<!-- END QUESTION -->

<br/>

---

## Question 1.4

In the previous question, many of the businesses had a common invalid postal code that was likely used to encode a MISSING postal code.  

Identify the single common missing postal code and assign it to `missing_postal_code`. Then create a `DataFrame`, `bus_missing`, to store only those businesses in `bus` that have `missing_postal_code` as their postal code.

In [ ]:
missing_postal_code = ...
bus_missing = ...

In [ ]:
grader.check("q14")

<!-- BEGIN QUESTION -->

<br/>

---

## Question 1.5

Examine the `invalid_zip_bus` `DataFrame` we computed in Question 1.3 and look at the businesses that DO NOT have the special MISSING ZIP code value. Some of the invalid postal codes are just the full 9-digit code rather than the first 5 digits. Create a new column named `postal5` in the original `bus` `DataFrame` which contains only the first 5 digits of the `postal_code` column.

Then, for any of the `postal5` ZIP code entries that were not a valid San Francisco ZIP code (according to `valid_zips`), the provided code will set the `postal5` value to `None`. 

**Hint:** You will find `str` accessors particularly useful. They allow you to use your usual Python string functions in tandem with a `DataFrame`. For example, if you wanted to use the `replace` function on every entry in a column of a `DataFrame` to change the letter 'a' to 'e', you could do so by writing `df['col_name'].str.replace('a', 'e')`. Think about the different ways you can extract the first 5 digits using regular Python code!

**Do not modify the provided code! Simply add your own code in place of the ellipses.**


In [ ]:
bus['postal5'] = None 
...

bus.loc[~bus['postal5'].isin(valid_zips), 'postal5'] = None
# Checking the corrected postal5 column
bus.loc[invalid_zip_bus.index, ['bid', 'name', 'postal_code', 'postal5']]

In [ ]:
grader.check("q15")

<!-- END QUESTION -->

<br/><br/>
<hr style="border: 5px solid #8a8c8c;" />
<hr style="border: 1px solid #ffcd00;" />

# Part 2: Investigate the Inspection Data 

Let's now turn to the inspection `DataFrame`. Earlier, we found that `ins` has 4 columns named 
`iid`, `score`, `date`, and `type`.  In this section, we determine the granularity of `ins` and investigate the kinds of information provided for the inspections. 

Let's start by looking again at the first 5 rows of `ins` to see what we have. 

In [ ]:
ins.head(5)

<br/>

---

## Question 2.1

We would like to extract `bid` from each row of the `ins` `DataFrame`. If we look carefully, the column `iid` of the `ins` `DataFrame` appears to be the composition of two numbers and the first number looks like a business id.  

Create a new column called `bid` in the `ins` Dataframe containing just the business id.  You will want to use `ins['iid'].str` operations to do this. (Python's in-built `split` method could come in use, read up on the documentation [here](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.split.html)!) Also, be sure to convert the type of this column to `int`. 

**Hint**: Similar to an earlier problem where we used `astype("string")` to convert a column to a string, here you should use `astype` to convert the `bid` column into type `int`. **No Python `for` loops or list comprehensions are allowed.** This is on the honor system since our autograder isn't smart enough to check, but if you're using `for` loops or list comprehensions, you're doing the HW incorrectly. 

In [ ]:
ins['bid'] = ...
ins.head(5)

In [ ]:
grader.check("q21")

<br/>

---

## Question 2.2

For this part, we're going to explore some new somewhat strange syntax that we haven't seen in lecture. Don't panic! If you're not sure what to do, try experimenting, Googling, and don't shy away from talking to the course staff.

For this problem we'll use the time component of the inspection data.  All of this information is given in the `date` column of the `ins` `DataFrame`. 


What is the type of the individual `ins['date']` entries? You may want to grab the very first entry and use the `type` function in Python. 

In [ ]:
ins_date_type = ...
ins_date_type

In [ ]:
grader.check("q22")

<br/>

---

## Question 2.3

Rather than the type you discovered in Question 2.2, we want each entry in `pd.TimeStamp` format. You might expect that the usual way to convert something from it current type to `TimeStamp` would be to use `astype`. You can do that, but the more typical way is to use `pd.to_datetime`. Using `pd.to_datetime`, create a new `ins['timestamp']` column containing `pd.Timestamp` objects.  These will allow us to do date manipulation with much greater ease. 

**No Python `for` loops or list comprehensions are allowed!**

Note: You may run into a UserWarning in case you do not specify the date format when using `pd.to_datetime`. To resolve this, consider using the following string '%m/%d/%Y %I:%M:%S %p' to specify the `format`.

In [ ]:
ins['timestamp'] = pd.to_datetime(ins['date'], format = ...
ins['timestamp']

In [ ]:
grader.check("q23")

<br/>

---

## Question 2.4

What are the earliest and latest dates in our inspection data?  

**Hint**: you can use `min` and `max` on dates of the correct type.

In [ ]:
earliest_date = ...
latest_date = ...
print("Earliest Date:", earliest_date)
print("Latest Date:", latest_date)

In [ ]:
grader.check("q24")

<br>

---

## Question 2.5
We probably want to examine the inspections by year. Create an additional `ins['year']` column containing just the year of the inspection.  Consider using `pd.Series.dt.year` to do this.

In case you're curious, the documentation for `TimeStamp` data can be found at [this link](https://pandas.pydata.org/docs/reference/api/pandas.Timestamp.html#pandas.Timestamp).

In [ ]:
ins['year'] = ins['timestamp'].dt.year
ins.head()

In [ ]:
grader.check("q25")

<br/><br/>
<hr style="border: 5px solid #8a8c8c;" />
<hr style="border: 1px solid #ffcd00;" />

# Congratulations! You have finished HW02!

### Submission Instructions

Below, you will see a cell. Running this cell will automatically generate a zip file with your autograded answers. Once you submit this file to the HW 2 Coding assignment on Gradescope, Gradescope will automatically submit a PDF file with your written answers to the HW 2 Written assignment. 

**Important**: Please check that your written responses were generated and submitted correctly to the HW 2 Written Assignment. 

**You are responsible for ensuring your submission follows our requirements and that the PDF for HW 2 written answers was generated/submitted correctly. We will not be granting regrade requests nor extensions to submissions that don't follow instructions.** If you encounter any difficulties with submission, please don't hesitate to reach out to staff prior to the deadline.

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export()